In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local').config('spark.driver.memory', '8g').appName('Check min date').getOrCreate()


In [2]:
df = spark.read.parquet(
    '/Users/gabriele.sabato/PycharmProjects/raw_data/price_elasticity_model_data/part-*',
    header=True)


In [3]:
df_d_item = spark.read.parquet('/Users/gabriele.sabato/PycharmProjects/raw_data/redshift_tables/d_item/*')


In [4]:
df.createOrReplaceTempView('model_data_table')

#df2 = spark.sql("SELECT MIN(update_date) from tmp_table")

In [5]:
df_d_item.createOrReplaceTempView('d_item_tmp')

In [7]:
df2 = spark.sql("SELECT * from model_data_table where update_date >= '2020-01-01'")


In [8]:
df4 = spark.sql ("SELECT item_code, item_main_category, item_sub_category_1, item_sub_category_2,item_parent_item_code from d_item_tmp")


In [10]:
df4.createOrReplaceTempView('d_item_filtered')
df2.createOrReplaceTempView('model_data_table_YTD')

In [13]:
df5 = spark.sql ("SELECT mdt.*, di.item_main_category, di.item_sub_category_1, di.item_sub_category_2, di.item_parent_item_code from model_data_table_YTD as mdt INNER JOIN d_item_filtered as di on di.item_code = mdt.item_code")

In [14]:
df5.show()

+------------------+-----------+--------------+------------+-----+------------------+-------------------+--------------------+---------------------+
|         item_code|update_date|delivery_weeks|  item_price|sales|item_main_category|item_sub_category_1| item_sub_category_2|item_parent_item_code|
+------------------+-----------+--------------+------------+-----+------------------+-------------------+--------------------+---------------------+
|000000001000000013| 2020-01-30|             1|       39.99|    0|            GARTEN|      GARTENZUBEHÖR|DEKORATION & ACCE...| 000000001000000013-P|
|000000001000000013| 2020-10-26|             0| 31.98999999|    0|            GARTEN|      GARTENZUBEHÖR|DEKORATION & ACCE...| 000000001000000013-P|
|000000001000000013| 2020-10-27|             0| 31.98999999|    0|            GARTEN|      GARTENZUBEHÖR|DEKORATION & ACCE...| 000000001000000013-P|
|000000001000000014| 2020-01-14|             0| 74.98999999|    0|            GARTEN|      GARTENZUBEHÖR|D

In [15]:
pd_merged = df5.toPandas()

In [16]:
pd_merged.to_pickle('/Users/gabriele.sabato/PycharmProjects/raw_data/DataFrames/data_model_cat_merged.pickle')

In [17]:
print(pd_merged.head())

            item_code update_date  delivery_weeks  item_price  sales  \
0  000000001000000013  2020-01-30               1       39.99      0   
1  000000001000000013  2020-10-26               0       31.99      0   
2  000000001000000013  2020-10-27               0       31.99      0   
3  000000001000000014  2020-01-14               0       74.99      0   
4  000000001000000020  2020-03-30               1       74.99      0   

  item_main_category item_sub_category_1       item_sub_category_2  \
0             GARTEN       GARTENZUBEHÖR  DEKORATION & ACCESSOIRES   
1             GARTEN       GARTENZUBEHÖR  DEKORATION & ACCESSOIRES   
2             GARTEN       GARTENZUBEHÖR  DEKORATION & ACCESSOIRES   
3             GARTEN       GARTENZUBEHÖR  DEKORATION & ACCESSOIRES   
4             GARTEN       GARTENZUBEHÖR  DEKORATION & ACCESSOIRES   

  item_parent_item_code  
0  000000001000000013-P  
1  000000001000000013-P  
2  000000001000000013-P  
3  000000001000000014-P  
4  0000000010000

In [ ]:
pd_merged.describe()

In [ ]:
print(pd_merged.dtypes)


In [ ]:
import pandas as pd

In [ ]:
pd_merged['update_date'] = pd.to_datetime(pd_merged['update_date'])

In [ ]:
pd_merged['week']= pd_merged['update_date'].dt.week
#pandadf.set_index('update_date', inplace=True)


In [ ]:
print(pd_merged)

In [ ]:
pd_merged.plot(x='update_date', y='item_price')

In [ ]:
pd_merged.to_pickle('/Users/gabriele.sabato/PycharmProjects/raw_data/DataFrames/somefile.pickle')